# <div align="center">*Acoustic Guitar Classifier*</div>

<div align="center">Want to know which kind of acoustic guitar you have? This machine learning classifier can detect dreadnought, jumbo, classical and resonator guitars with 87% accuracy. If you `upload a clear and complete picture` of your guitar, you increase the chance that the classifier will get it right.</div>

<div align="center">Simply try it out :-)</div>

In [1]:
# To turn your notebook into a webapp, install voila which will only show markdown and outputs and hide the rest

# Install voila library and application
#!pip install voila

# Connect voila to this notebook
#!jupyter serverextension enable voila --sys-prefix

In [1]:
# Import only the necessary libraries for deployment
from utils import *
from fastai2.vision.widgets import *

In [2]:
# Set the path to wherever you have stored your trained classifier
path = Path()

In [3]:
# Load the trained classifier
learn_inf = load_learner(path/'export.pkl')

In [4]:
# Define all iPython widgets that you want to use

# We need a button for uploading images
btn_upload = widgets.FileUpload()
# A button to run the classification
btn_run = widgets.Button(description='Classify image')
# Another button to show all predictions
btn_show = widgets.Button(description='Show others')
# We want to show the uploaded image as an output
out_pl = widgets.Output()
# And finally we need to display the prediction as a label
lbl_pred = widgets.Label()
# Add more labels for other predictions
lbl_pred_a = widgets.Label()
lbl_pred_b = widgets.Label()
lbl_pred_c = widgets.Label()

In [5]:
# Create a string to expand the label below
lbl_ext = ' guitar'

In [6]:
# Since the run button doesn't do anything, we need to define an event handler 
def on_click_classify(change):
    # Turn the uploaded image into a PIL image file
    img = PILImage.create(btn_upload.data[-1])
    # Clear all previous outputs
    out_pl.clear_output()
    # Display the uploaded image
    with out_pl: display(img.to_thumb(224,224))
    # Run the classifier on the image
    pred,pred_idx,probs = learn_inf.predict(img)
    # Print out the classification results
    lbl_pred.value = f'Prediction: {(pred + lbl_ext).title()}; Probability: {probs[pred_idx]:.04f}'

In [7]:
# Attach the event handler to the run button
btn_run.on_click(on_click_classify)

In [8]:
# Define an event handler for the 
def on_click_show_others(change):
    # Turn the uploaded image into a PIL image file
    img = PILImage.create(btn_upload.data[-1])
    # Run the classifier on the image
    pred,pred_idx,probs = learn_inf.predict(img)
    # Zip class names and probabilities, convert into a dataframe and sort by descending probability
    res = pd.DataFrame(zip(learn_inf.dls.vocab, probs.numpy())).sort_values(by=1, ascending=False)
    # Print out the predicted probabilities for the three remaining classes
    lbl_pred_a.value = f'{(res.iloc[1,0] + lbl_ext).title()}: {res.iloc[1,1]:.06f}'
    lbl_pred_b.value = f'{(res.iloc[2,0] + lbl_ext).title()}: {res.iloc[2,1]:.06f}'
    lbl_pred_c.value = f'{(res.iloc[3,0] + lbl_ext).title()}: {res.iloc[3,1]:.06f}'

In [9]:
# Attach the event handler to the show others button
btn_show.on_click(on_click_show_others)

In [10]:
# Put everything together in a virtual box (note that the order of the components matters)
VBox([widgets.Label('Upload a picture of your guitar!'), 
      btn_upload, btn_run, out_pl, lbl_pred, btn_show, lbl_pred_a, lbl_pred_b, lbl_pred_c], 
      layout=Layout(align_items='center'))